In [1]:
import pandas as pd
train_data = pd.read_csv('/home/cs-folq1/rds/rds-t2-cspp025-5bF3aEHVmLU/cs-folq1/test1/cw2_train.csv')
dev_data = pd.read_csv('/home/cs-folq1/rds/rds-t2-cspp025-5bF3aEHVmLU/cs-folq1/test1/cw2_dev.csv')

train_data

,Unnamed: 0,Sent1,Sent2,SimScore
0,0,"U.S., EU Widen Sanctions On Russia","U.S., EU Boost Sanctions On Russia",1.00
1,1,The lawyers advised the judges .,The lawyers advised the judges behind the acto...,0.79
2,2,Man kills 4 in Calif. before police shoot him ...,Police: Gunman killed 6 in California shootings,0.40
3,3,Someone is playing a piano.,A man is playing a guitar.,0.24
4,4,In an E-mail statement to the Knoxville News S...,I am not giving any consideration to resignati...,0.80
...,...,...,...,...
11493,11493,A man is playing piano.,A man is laying on the ground.,0.15
11494,11494,"The doctors resigned , or the secretaries supp...",The doctors resigned .,0.50
11495,11495,The artist contacted the banker .,The banker contacted the artist by the student .,0.29
11496,11496,"While the professors arrived , the student wai...",The professors arrived .,0.61


In [2]:
import torch
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
device

'cpu'

In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [5]:
model = SentenceTransformer('./saved_models/sbert1')

In [3]:
# Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens', device = device)

In [4]:
# Define your train examples.
train_examples = [InputExample(texts=[item[1]['Sent1'], item[1]['Sent2']], label=item[1]['SimScore']) 
                               for item in train_data.iterrows()]

# Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [6]:
from sentence_transformers import evaluation

sentences1 = [item[1]['Sent1'] for item in dev_data.iterrows()]
sentences2 = [item[1]['Sent2'] for item in dev_data.iterrows()]
scores = [item[1]['SimScore'] for item in dev_data.iterrows()]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

def eval_callback(score, epoch, steps):
    print('Epoch {}, step {}: score {:.4f}'.format(epoch, steps, score))

In [39]:
# Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100,
          evaluator=evaluator, evaluation_steps=200, callback=eval_callback)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

Epoch 0, step 200: score 0.7925
Epoch 0, step 400: score 0.8586
Epoch 0, step 600: score 0.8829
Epoch 0, step -1: score 0.8893


Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

Epoch 1, step 200: score 0.8945
Epoch 1, step 400: score 0.8976
Epoch 1, step 600: score 0.9011
Epoch 1, step -1: score 0.9007


...

Epoch 2, step 200: score 0.9022
Epoch 2, step 400: score 0.9028
Epoch 2, step 600: score 0.9042
Epoch 2, step -1: score 0.9059


Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

Epoch 3, step 200: score 0.9065
Epoch 3, step 400: score 0.9064
Epoch 3, step 600: score 0.9054
Epoch 3, step -1: score 0.9061


Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

Epoch 4, step 200: score 0.9055
Epoch 4, step 400: score 0.9064
Epoch 4, step 600: score 0.9065
Epoch 4, step -1: score 0.9067


In [7]:
from sentence_transformers import util

# Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarity
pred_scores = util.pytorch_cos_sim(embeddings1, embeddings2).diag().cpu()

KeyboardInterrupt: 

In [16]:
gold_scores = torch.Tensor(scores)
print(gold_scores.max(), gold_scores.min())
print(pred_scores.max(), pred_scores.min())

tensor(1.) tensor(0.)
tensor(1.0000) tensor(-0.2296)


In [36]:
order = torch.argsort(torch.abs(pred_scores - gold_scores))
k = -9
print('Gold: {:.4f}, predicted: {:.4f}'.format(gold_scores[order[k]], pred_scores[order[k]]))
print(sentences1[order[k]])
print(sentences2[order[k]])

Gold: 0.3200, predicted: 0.7736
Israeli Leader Puts Off Evicting Settlers From Hebron House
Israeli army orders settlers out of Hebron house


In [37]:
model.save('./test2_saved_model')

In [39]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [13]:
def compute_cosine_sim(model, sent1, sent2):
    emb = model.encode([sent1, sent2], convert_to_tensor=True)
    return util.pytorch_cos_sim(emb, emb)[0,1].item()

In [17]:
compute_cosine_sim(model, 'Hello world', 'Bye world')

0.501793622970581